In [ ]:
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import os
from lora_diffusion import tune_lora_scale, patch_pipe
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
imagenet_templates_small = [
    "realistic photo of {}",
    "realistic render of {}",
    "painting of {}",
    "{}",
    "anime styled {}",
    "vector art style {}",
]
model_id = "stabilityai/stable-diffusion-2-1-base"
tensors_path = "/Downloads/sex/step_100.safetensors"
tensors_path = "/Projects/Personal/leap-sd/training/tools/lora/original_loras/vol.safetensors"
# tensors_path = "/Projects/Personal/leap-sd/training/lora_dataset_creator/lora_dataset/dragon/models/step_1000.safetensors"

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(
    "cuda"
)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

In [ ]:
def display_grid(images, rows, cols, titles):
    fig, axs = plt.subplots(rows, cols, figsize=(7, 6))
    for i, ax in enumerate(axs.flat):
        ax.imshow(images[i])
        ax.axis('off')
        ax.set_title(titles[i])
    plt.show()
    
def test_gen():
    torch.manual_seed(0)
    images = [pipe(prompt.format("<s1>"), num_inference_steps=25, guidance_scale=9).images[0] for prompt in imagenet_templates_small]
    display_grid(images, 2, 3, imagenet_templates_small)

In [ ]:
patch_pipe(
    pipe,
    tensors_path,
    patch_text=True,
    patch_ti=True,
    patch_unet=True,
)

In [ ]:
tune_lora_scale(pipe.unet, 1)
tune_lora_scale(pipe.text_encoder, 1)
test_gen()

In [ ]:
tune_lora_scale(pipe.unet, 0.2)
tune_lora_scale(pipe.text_encoder, 0.2)
test_gen()